In [1]:
# Importing the Modules
import dash
import base64
import datetime
import numpy as np
import pandas as pd
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import mean_squared_error,r2_score
from win32api import GetSystemMetrics
width = GetSystemMetrics(0)
height = GetSystemMetrics(1)

In [2]:
# Reading the dataset
dataset = pd.read_csv('Climate_data.csv')
dataset.head()

,datetime_utc,conds,dewptm,fog,hail,heatindexm,hum,precipm,pressurem,rain,...,vism,wdird,wdire,wgustm,windchillm,wspdm,month,day,year,time
0,19961101-11:00,Smoke,9,0,0,NaN,27,NaN,1010.0,0,...,5.0,280,West,NaN,NaN,7.4,11,1,1996,11:00
1,19961101-12:00,Smoke,10,0,0,NaN,32,NaN,-9999.0,0,...,5.0,0,North,NaN,NaN,7.4,11,1,1996,12:00
2,19961101-13:00,Smoke,11,0,0,NaN,44,NaN,-9999.0,0,...,5.0,0,North,NaN,NaN,7.4,11,1,1996,13:00
3,19961101-14:00,Smoke,10,0,0,NaN,41,NaN,1010.0,0,...,2.0,0,North,NaN,NaN,7.4,11,1,1996,14:00
4,19961101-16:00,Smoke,11,0,0,NaN,47,NaN,1011.0,0,...,1.2,0,North,NaN,NaN,0.0,11,1,1996,16:00


In [3]:
# Loading the image of Table Backgound
table_image = base64.b64encode(open('table_img.png', 'rb').read())


# Loading the image of Team Members
team_members = base64.b64encode(open('team_members.png', 'rb').read())

In [4]:
# App initialization
app = dash.Dash()

In [18]:
# style for Rows
style_R= {
    'color':'#111111',
    'fontSize': '25px',
    'fontFamily': 'Comic San MS',
    'height': '80px',
}

# style for Header
style_H= {
    'color':'#4d4d80',
    'fontSize': '30px',
    'fontFamily': 'Comic San MS',
    'height':'60px',
    'text-align':'left',
    'width':'50px'
}


In [19]:
# Setting layout for web page
app.layout = html.Div([
    
    # Background image div
    html.Div(id='background',style= {'width':'100%', 'height':'100%'}),
    
    # Date Picker Componnets div
    html.Div([dcc.DatePickerSingle(id='date-picker', 
                                   date=datetime.datetime.now(),
                                   min_date_allowed=datetime.datetime(1996, 11, 1), 
                                   display_format='DD MMM, YYYY',
                                   placeholder='Select a date',
                                   reopen_calendar_on_clear=True)],
             style={'margin-top':20, 'margin-left':50}),
    
    # Table background image div
    html.Div(html.Img(src='data:image/png;base64,{}'.format(table_image.decode()), 
                      style={'width':350, 'height':400}),
             style= {'width':0, 'height':0, 'margin-top':40, 'margin-left':50}),
    
    # Table for Predicted Temperature div
    html.Div(id='temp_table',
             style={'width': '49%', 'display': 'inline-block','margin-left': 60}),
    
    # Graph of Weekly's Temperature Variation div
    html.Div(id='Graph',
             style={'width': '40%','height':'30%', 'float': 'right', 'display': 'inline-block','margin-right': 50}),
    
    # Image of Team Memgers div
    html.Img(src='data:image/png;base64,{}'.format(team_members.decode()), 
             style={'width':width, 'height':500,'margin-top':80}),
    
    # Dash Component for Changing Background
    dcc.Interval(id='interval-component', interval=1*4000, n_intervals=0)
])


# callback function for date picker object to predict temperature and showing Html Table 
@app.callback(Output('temp_table', 'children'), [Input('date-picker', 'date')])
def update_output(date):
    if date is not None:
        year = int(date[:4])
        month = int(date[5:7])
        day = int(date[8:10])
        d = list()
        d.append(datetime.date(year,month,day))
        for i in range(1,4):
            d.append(d[0] + datetime.timedelta(days=i))
        temp = list()
        for i in range(4):
            temp.append(model_training(d[i]))
            
        # returning a Html Table of predicted temperature    
        return html.Table(
            [html.Tr([html.Th("Date"), html.Th("Temperature")] ,style=style_H)] 
            +
            [html.Tr([html.Td("{:%b %d, %A }".format(d[i])), html.Td(str(temp[i])+'° C', style={'text-align':'right'})],
                     style=style_R) for i in range(4)], 
            style={'width':'30px'})

    
# callback function for Graph     
@app.callback(Output('Graph', 'children'), [Input('date-picker', 'date')])
def update_output(date):
    if date is not None:
        year = int(date[:4])
        month = int(date[5:7])
        day = int(date[8:10])
        d = list()
        for i in range(-14,15,7):
            d.append(datetime.date(year,month,day) + datetime.timedelta(days=i))
        temp = list()
        for i in d:
            temp.append(model_training(i))
        return dcc.Graph(id='example-graph',
                         figure={'data':[{'x': d, 'y': temp, 'type': 'line'}],                           
                                 'layout': {'title':  'WEEKLY TEMPERATURE VARIATION',
                                            'plot_bgcolor': '#DBEC9A',
                                            'paper_bgcolor': '#E9F3C2',
                                            'font':{ 'color': '#0041f2'},
                                            'xaxis':{'title': 'DATE'},
                                            'yaxis':{'title':'TEMPERATURE'}
                                           }
                                }
                        )
    

# Callback function for changing Image    
@app.callback(Output('background', 'children'), [Input('interval-component', 'n_intervals')])
def update_graph_live(n):
    image_filename = 'bg_'+str(int(n%16))+'.png'
    encoded_image = base64.b64encode(open(image_filename, 'rb').read())
    return [html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), 
                     style={'width':width, 'height':height})]

CantHaveMultipleOutputs: 
You have already assigned a callback to the output
with ID "temp_table" and property "children". An output can only have
a single callback function. Try combining your inputs and
callback functions together into one function.


In [20]:

# Training the model for weather predictions
def model_training(date):
    year = date.year
    month = date.month
    day = date.day
    
    # Selecting Appropriate dataset for prediction
    df = dataset[(dataset['month']==month) & (dataset['day']<=day) & (dataset['day']>=day-3)]
    
    # Independent variable data selection
    X = df.iloc[:,[2,6,8,19]].values
    
    # Dependent variable data selection
    y = df.iloc[:,11].values
    
    # spliting data into train and test
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1)
    
    # Model - 1
    # Creating a Simple Linear Regressor and fitting dataset
    lin_reg = LinearRegression()
    lin_reg.fit(X_train, y_train) 
    
    # Model - 2
    # Creating Decision tree Regressor and fitting dataset
    dec_reg = DecisionTreeRegressor(random_state = 0)
    dec_reg.fit(X_train, y_train)
    
    # Model - 3
    # Fitting Random Forest Regression to the dataset
    for_reg = RandomForestRegressor(n_estimators = 10, random_state = 0)
    for_reg.fit(X_train, y_train)

    # predicting on test data
    lin_pred = np.round(lin_reg.predict(X_test)).astype(int)
    dec_pred = np.round(dec_reg.predict(X_test)).astype(int)
    for_pred = np.round(for_reg.predict(X_test)).astype(int)
    
    # Creating datframe of Comparision metrics
    cm = pd.DataFrame({'Actual':y_test, 'Linear_Pred': lin_pred, 'Decision_Pred': dec_pred, 'Forest_Pred': for_pred})

    stat = pd.DataFrame({'Model':['Linear Regressor','Decision Regressor','Forest Regressor'],
                         'Mean squared error': [mean_squared_error(y_test,lin_pred),
                                                mean_squared_error(y_test,dec_pred),
                                                mean_squared_error(y_test,for_pred)],
                        'Root mean squared error': [np.sqrt(mean_squared_error(y_test,lin_pred)),
                                                    np.sqrt(mean_squared_error(y_test,dec_pred)),
                                                    np.sqrt(mean_squared_error(y_test,for_pred))],
                        'R_2 statics': [r2_score(y_test, lin_pred),
                                        r2_score(y_test, dec_pred),
                                        r2_score(y_test, for_pred)]})
    stat = stat.set_index('Model')
    
    print('\n\n',date,'\n----------------------')
    print(cm.head())
    print('\n',stat)
    
    # Calculating the mean values of Independent variables Data
    
    dew=np.mean(X[:,0])                         # Dew point 
    hum=np.mean(X[:,1])                         # Humidty   
    press=np.mean(X[:,2])                       # Pressure 
    windsp=np.mean(X[:,3])                      # Wind speed

    # Creating a 2D numpy array of shape(1, 4)
    test = np.array([dew,hum,press,windsp]).reshape((1,4))
    
    # Predicting a new result
    temperature = for_reg.predict(test)
    
    
    # returning the predicted temperature
    return int(round(*temperature))
    

In [21]:
# main 
if __name__ == "__main__":
    app.run_server(debug=False)

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
